# Getting temperature dependent data.

# Not an exercise for students! 
Author:

J. Angevaare // <j.angevaare@nikhef.nl> // 2020-06-11

This is mostly for getting the data in the right place. May not be too interesting for the students.

## Temperature calibrations:
Dates:
  - 2016-04-25 to
  - 2016-05-09 

and:
  - 2016-01-07 to
  - 2016-01-22

In [1]:
import socket

In [2]:
import os
import numpy as np

In [3]:
def on_stoomboot():
    '''Check that you can do stuff here otherwise raise an error'''
    host = socket.gethostname()
    if not 'nikhef' in host:
        raise ValueError(f'You are not on stoomboot but on {host}. '
                          'You can not do this operation! Perhaps continue below')
        

In [4]:
base = '/dcache/xenon/tmons/Modulation/processed/'

In [5]:
all_files = []

In [6]:
on_stoomboot()
for d in range(7,23):
    if d >= 10:
        date = f'201601{d}'
    else:
        date = f'2016010{d}'
    files = !ls $base | grep $date
    all_files += files

ValueError: You are not on stoomboot but on DESKTOP-7U5FNIR. You can not do this operation! Perhaps continue below

In [7]:
all_files

['mx_n_20160107_1209',
 'mx_n_20160108_1934',
 'mx_n_20160113_1228',
 'mx_n_20160116_0940',
 'mx_n_20160119_0803',
 'mx_n_20160122_1556']

In [8]:
on_stoomboot()
for d in range(25,31):
    date = f'201604{d}'
    files = !ls $base | grep $date
    all_files += files

In [9]:
all_files

['mx_n_20160107_1209',
 'mx_n_20160108_1934',
 'mx_n_20160113_1228',
 'mx_n_20160116_0940',
 'mx_n_20160119_0803',
 'mx_n_20160122_1556',
 'mx_n_20160425_0752',
 'mx_n_20160426_0736',
 'mx_n_20160428_0755',
 'mx_n_20160430_0840']

In [10]:
on_stoomboot()
for d in range(0,10):
    date = f'2016050{d}'
    files = !ls $base | grep $date
    all_files += files

In [11]:
all_files

['mx_n_20160107_1209',
 'mx_n_20160108_1934',
 'mx_n_20160113_1228',
 'mx_n_20160116_0940',
 'mx_n_20160119_0803',
 'mx_n_20160122_1556',
 'mx_n_20160425_0752',
 'mx_n_20160426_0736',
 'mx_n_20160428_0755',
 'mx_n_20160430_0840',
 'mx_n_20160502_0752',
 'mx_n_20160504_0846',
 'mx_n_20160506_1521',
 'mx_n_20160509_0805']

In [30]:
first_chuncks = []
for f in all_files:
    p = os.path.join(base, f)
    f = !ls $p | grep 000000
    first_chuncks.append(os.path.join(p,f[0]))

In [13]:
first_chuncks

['/dcache/xenon/tmons/Modulation/processed/mx_n_20160107_1209/mx_n_20160107_1209_000000.root',
 '/dcache/xenon/tmons/Modulation/processed/mx_n_20160108_1934/mx_n_20160108_1934_000000.root',
 '/dcache/xenon/tmons/Modulation/processed/mx_n_20160113_1228/mx_n_20160113_1228_000000.root',
 '/dcache/xenon/tmons/Modulation/processed/mx_n_20160116_0940/mx_n_20160116_0940_000000.root',
 '/dcache/xenon/tmons/Modulation/processed/mx_n_20160119_0803/mx_n_20160119_0803_000000.root',
 '/dcache/xenon/tmons/Modulation/processed/mx_n_20160122_1556/mx_n_20160122_1556_000000.root',
 '/dcache/xenon/tmons/Modulation/processed/mx_n_20160425_0752/mx_n_20160425_0752_000000.root',
 '/dcache/xenon/tmons/Modulation/processed/mx_n_20160426_0736/mx_n_20160426_0736_000000.root',
 '/dcache/xenon/tmons/Modulation/processed/mx_n_20160428_0755/mx_n_20160428_0755_000000.root',
 '/dcache/xenon/tmons/Modulation/processed/mx_n_20160430_0840/mx_n_20160430_0840_000000.root',
 '/dcache/xenon/tmons/Modulation/processed/mx_n_20

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numba
import uproot3
import tqdm

In [17]:
file = uproot3.open(first_chuncks[0])
tree = file['T;2']
data = tree.pandas.df()

In [20]:
data[['channel', 'integral','time', 'temp']]

,channel,integral,time,temp
entry,,,,
0,4,643.492432,3.661157e+09,28.7
1,1,4089.272949,3.661157e+09,28.7
2,7,647.649597,3.661157e+09,28.7
3,1,502.088806,3.661157e+09,28.7
4,2,682.318359,3.661157e+09,28.7
...,...,...,...,...
3559521,6,486.438202,3.661159e+09,28.7
3559522,2,503.663025,3.661159e+09,28.7
3559523,3,507.591431,3.661159e+09,28.7


In [21]:
!mkdir ../data/temperature

In [46]:
sources = {'bg':[0,1], 
           'ti44':[2,3], 
           'co60':[4,5], 
           'cs137':[6,7]}
store_columns = ['channel', 'integral','time', 'temp']
for root_file in tqdm.tqdm(first_chuncks):
    if not '.root' in root_file:
        # This is not a rootfile
        continue
    else:
        name = root_file.split('/')[-1].split('.root')[0]
    file = uproot.open(root_file)
    tree = file['T;2']
    data = tree.pandas.df()
    print(f'{name}\tT = {np.mean(data["temp"])} +/- {np.std(data["temp"])}')
    for source, channels in sources.items():
        mask = ( 
            (data['channel'] == channels[0] ) | (data['channel'] == channels[1] ) 
            & (data['istestpulse'] == 0) 
            & (data['error'] == 0) 
        )
        save_dir = f'../data/temperature/{source}'
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        save_name = f'{save_dir}/{source}_{name}.zip'
#         data['tot_time'] = data['time'].max() - data['time'].min()
#         data['dtemp'] = np.std(data['temp'])
#         data['temp'] = np.mean(data['temp'])
        data[mask][store_columns].to_parquet(save_name,index=False)
    # Double check that we free up memory
    del data, mask

  0%|          | 0/14 [00:00<?, ?it/s]

mx_n_20160107_1209_000000	T = 28.70580900378319 +/- 0.023391362793922142


  7%|▋         | 1/14 [00:04<00:58,  4.49s/it]

mx_n_20160108_1934_000000	T = 23.494752756406317 +/- 0.02229777554007703


 14%|█▍        | 2/14 [00:08<00:53,  4.42s/it]

...

 86%|████████▌ | 12/14 [00:52<00:08,  4.31s/it]

mx_n_20160506_1521_000000	T = 27.375153624803488 +/- 0.0432122107653618


 93%|█████████▎| 13/14 [00:56<00:04,  4.38s/it]

mx_n_20160509_0805_000000	T = 23.0 +/- 0.0


100%|██████████| 14/14 [01:00<00:00,  4.35s/it]


# Alright let's continue to 7.) we are done here